# Hodina 12: Neinformované prohledávání (BFS, DFS) 🔍

## Přehled lekce

V této hodině se naučíme:
- Implementovat základní prohledávací algoritmy (BFS, DFS)
- Vizualizovat průběh prohledávání s neuronovými sítěmi
- Použít transformery pro predikci nejlepší strategie
- Vytvořit interaktivní simulace prohledávání

---

## 1. Nastavení prostředí a instalace knihoven

In [ ]:
# Instalace potřebných knihoven
!pip install torch torchvision transformers gradio networkx matplotlib numpy
!pip install plotly ipywidgets pygame celluloid

import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from matplotlib.animation import FuncAnimation
import networkx as nx
from collections import deque, defaultdict
import time
from IPython.display import HTML, display, clear_output
import plotly.graph_objects as go
import gradio as gr
from transformers import pipeline
import json
from celluloid import Camera
import random

# Nastavení zobrazení
plt.style.use('seaborn-v0_8-darkgrid')
np.random.seed(42)
torch.manual_seed(42)

## 2. Breadth-First Search (BFS) - Prohledávání do šířky 📊

BFS systematicky prozkoumává všechny uzly ve stejné vzdálenosti od startu.

In [ ]:
# Vizualizovaná implementace BFS
class VisualBFS:
    def __init__(self, graph):
        self.graph = graph
        self.visited = set()
        self.queue = deque()
        self.path = {}
        self.search_order = []
        
    def search(self, start, goal):
        """BFS s ukládáním kroků pro vizualizaci"""
        self.queue.append(start)
        self.visited.add(start)
        self.path[start] = None
        
        steps = []
        
        while self.queue:
            current = self.queue.popleft()
            self.search_order.append(current)
            
            # Uložení kroku pro vizualizaci
            steps.append({
                'current': current,
                'queue': list(self.queue),
                'visited': set(self.visited),
                'found': current == goal
            })
            
            if current == goal:
                return self._reconstruct_path(goal), steps
            
            for neighbor in self.graph[current]:
                if neighbor not in self.visited:
                    self.visited.add(neighbor)
                    self.queue.append(neighbor)
                    self.path[neighbor] = current
        
        return None, steps
    
    def _reconstruct_path(self, goal):
        path = []
        current = goal
        while current is not None:
            path.append(current)
            current = self.path[current]
        return path[::-1]

# Vytvoření testovacího grafu
def create_sample_graph():
    graph = {
        'A': ['B', 'C'],
        'B': ['A', 'D', 'E'],
        'C': ['A', 'F'],
        'D': ['B'],
        'E': ['B', 'F'],
        'F': ['C', 'E', 'G'],
        'G': ['F']
    }
    return graph

# Animovaná vizualizace BFS
def animate_bfs(graph, start='A', goal='G'):
    bfs = VisualBFS(graph)
    path, steps = bfs.search(start, goal)
    
    # Vytvoření NetworkX grafu pro vizualizaci
    G = nx.Graph(graph)
    pos = nx.spring_layout(G, seed=42)
    
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))
    camera = Camera(fig)
    
    for i, step in enumerate(steps):
        ax1.clear()
        ax2.clear()
        
        # Graf
        node_colors = []
        for node in G.nodes():
            if node == step['current']:
                node_colors.append('red')
            elif node in step['visited']:
                node_colors.append('lightblue')
            elif node in step['queue']:
                node_colors.append('yellow')
            else:
                node_colors.append('lightgray')
        
        nx.draw(G, pos, ax=ax1, node_color=node_colors, 
                node_size=1000, with_labels=True, font_size=16)
        ax1.set_title(f'BFS - Krok {i+1}: Aktuální uzel = {step["current"]}', 
                     fontsize=14)
        
        # Fronta a navštívené uzly
        ax2.text(0.1, 0.8, f"Fronta: {step['queue']}", fontsize=12)
        ax2.text(0.1, 0.6, f"Navštívené: {sorted(step['visited'])}", fontsize=12)
        ax2.text(0.1, 0.4, f"Aktuální: {step['current']}", fontsize=12, 
                color='red', weight='bold')
        
        if step['found']:
            ax2.text(0.1, 0.2, "✅ CÍL NALEZEN!", fontsize=16, 
                    color='green', weight='bold')
        
        ax2.axis('off')
        ax2.set_xlim(0, 1)
        ax2.set_ylim(0, 1)
        
        camera.snap()
    
    animation_obj = camera.animate(interval=1000)
    plt.close()
    
    return animation_obj, path

# Spuštění animace
graph = create_sample_graph()
print("🔍 Vizualizace BFS algoritmu:")
print("Graf:", graph)

anim, path = animate_bfs(graph)
HTML(anim.to_jshtml())

## 3. Depth-First Search (DFS) - Prohledávání do hloubky 🏔️

DFS prozkoumává co nejhlouběji, než se vrátí zpět.

In [ ]:
# Vizualizovaná implementace DFS
class VisualDFS:
    def __init__(self, graph):
        self.graph = graph
        self.visited = set()
        self.stack = []
        self.path = {}
        self.search_order = []
        
    def search(self, start, goal):
        """DFS s ukládáním kroků pro vizualizaci"""
        self.stack.append(start)
        self.path[start] = None
        
        steps = []
        
        while self.stack:
            current = self.stack.pop()
            
            if current in self.visited:
                continue
                
            self.visited.add(current)
            self.search_order.append(current)
            
            # Uložení kroku
            steps.append({
                'current': current,
                'stack': list(self.stack),
                'visited': set(self.visited),
                'found': current == goal
            })
            
            if current == goal:
                return self._reconstruct_path(goal), steps
            
            # Přidání sousedů v obráceném pořadí (pro konzistentní průchod)
            for neighbor in reversed(self.graph[current]):
                if neighbor not in self.visited:
                    self.stack.append(neighbor)
                    if neighbor not in self.path:
                        self.path[neighbor] = current
        
        return None, steps
    
    def _reconstruct_path(self, goal):
        path = []
        current = goal
        while current is not None:
            path.append(current)
            current = self.path.get(current)
        return path[::-1]

# Porovnání BFS a DFS
def compare_search_algorithms(graph, start='A', goal='G'):
    # BFS
    bfs = VisualBFS(graph)
    bfs_path, bfs_steps = bfs.search(start, goal)
    
    # DFS
    dfs = VisualDFS(graph)
    dfs_path, dfs_steps = dfs.search(start, goal)
    
    # Vizualizace porovnání
    fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(15, 12))
    
    G = nx.Graph(graph)
    pos = nx.spring_layout(G, seed=42)
    
    # BFS vizualizace
    bfs_colors = ['lightblue' if node in bfs.visited else 'lightgray' 
                  for node in G.nodes()]
    nx.draw(G, pos, ax=ax1, node_color=bfs_colors, 
            node_size=1000, with_labels=True)
    ax1.set_title('BFS - Navštívené uzly', fontsize=14)
    
    # BFS cesta
    if bfs_path:
        path_edges = [(bfs_path[i], bfs_path[i+1]) 
                     for i in range(len(bfs_path)-1)]
        nx.draw_networkx_edges(G, pos, ax=ax1, edgelist=path_edges, 
                              edge_color='red', width=3)
    
    # DFS vizualizace
    dfs_colors = ['lightgreen' if node in dfs.visited else 'lightgray' 
                  for node in G.nodes()]
    nx.draw(G, pos, ax=ax2, node_color=dfs_colors, 
            node_size=1000, with_labels=True)
    ax2.set_title('DFS - Navštívené uzly', fontsize=14)
    
    # DFS cesta
    if dfs_path:
        path_edges = [(dfs_path[i], dfs_path[i+1]) 
                     for i in range(len(dfs_path)-1)]
        nx.draw_networkx_edges(G, pos, ax=ax2, edgelist=path_edges, 
                              edge_color='red', width=3)
    
    # Statistiky BFS
    ax3.text(0.1, 0.8, "BFS Statistiky:", fontsize=16, weight='bold')
    ax3.text(0.1, 0.6, f"Počet kroků: {len(bfs_steps)}", fontsize=12)
    ax3.text(0.1, 0.5, f"Délka cesty: {len(bfs_path) if bfs_path else 'Nenalezena'}", fontsize=12)
    ax3.text(0.1, 0.4, f"Pořadí návštěvy: {bfs.search_order}", fontsize=10)
    ax3.text(0.1, 0.3, f"Cesta: {' → '.join(bfs_path) if bfs_path else 'Nenalezena'}", fontsize=12)
    ax3.axis('off')
    
    # Statistiky DFS
    ax4.text(0.1, 0.8, "DFS Statistiky:", fontsize=16, weight='bold')
    ax4.text(0.1, 0.6, f"Počet kroků: {len(dfs_steps)}", fontsize=12)
    ax4.text(0.1, 0.5, f"Délka cesty: {len(dfs_path) if dfs_path else 'Nenalezena'}", fontsize=12)
    ax4.text(0.1, 0.4, f"Pořadí návštěvy: {dfs.search_order}", fontsize=10)
    ax4.text(0.1, 0.3, f"Cesta: {' → '.join(dfs_path) if dfs_path else 'Nenalezena'}", fontsize=12)
    ax4.axis('off')
    
    plt.tight_layout()
    plt.show()
    
    return bfs_path, dfs_path

# Spuštění porovnání
print("📊 Porovnání BFS a DFS:")
bfs_path, dfs_path = compare_search_algorithms(graph)

## 4. Neuronové sítě pro predikci strategie prohledávání 🧠

Naučíme neuronovou síť předpovídat, který algoritmus bude efektivnější.

In [ ]:
# Neuronová síť pro predikci optimální strategie
class SearchStrategyPredictor(nn.Module):
    def __init__(self, input_dim=10, hidden_dim=64):
        super(SearchStrategyPredictor, self).__init__()
        
        self.encoder = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.ReLU(),
            nn.BatchNorm1d(hidden_dim),
            nn.Dropout(0.3),
            nn.Linear(hidden_dim, hidden_dim // 2),
            nn.ReLU(),
            nn.Linear(hidden_dim // 2, 2)  # 2 výstupy: BFS nebo DFS
        )
    
    def forward(self, x):
        return self.encoder(x)

# Generování trénovacích dat
def generate_graph_features(n_nodes, density=0.3):
    """Generuje náhodný graf a jeho příznaky"""
    # Vytvoření náhodného grafu
    G = nx.erdos_renyi_graph(n_nodes, density)
    
    # Extrakce příznaků
    features = [
        n_nodes,                                    # Počet uzlů
        G.number_of_edges(),                        # Počet hran
        nx.density(G),                              # Hustota
        nx.average_clustering(G),                   # Průměrný clustering
        nx.diameter(G) if nx.is_connected(G) else 0,  # Průměr grafu
        len(list(nx.connected_components(G))),     # Počet komponent
        np.mean([d for n, d in G.degree()]),       # Průměrný stupeň
        np.std([d for n, d in G.degree()]),        # Std stupně
        1 if nx.is_tree(G) else 0,                 # Je to strom?
        1 if nx.is_connected(G) else 0             # Je souvislý?
    ]
    
    return G, np.array(features)

def evaluate_search_performance(G, start=0, goal=None):
    """Vyhodnotí výkon BFS a DFS na grafu"""
    if goal is None:
        nodes = list(G.nodes())
        goal = nodes[-1] if nodes else 0
    
    # Převod na dict reprezentaci
    graph_dict = {n: list(G.neighbors(n)) for n in G.nodes()}
    
    # BFS
    bfs = VisualBFS(graph_dict)
    bfs_path, bfs_steps = bfs.search(start, goal)
    bfs_steps_count = len(bfs_steps)
    
    # DFS
    dfs = VisualDFS(graph_dict)
    dfs_path, dfs_steps = dfs.search(start, goal)
    dfs_steps_count = len(dfs_steps)
    
    # Label: 0 = BFS lepší, 1 = DFS lepší
    label = 0 if bfs_steps_count <= dfs_steps_count else 1
    
    return label, bfs_steps_count, dfs_steps_count

# Generování datasetu
print("🎲 Generování trénovacích dat...")
X_train = []
y_train = []

for _ in range(500):
    n_nodes = np.random.randint(5, 20)
    density = np.random.uniform(0.1, 0.5)
    
    try:
        G, features = generate_graph_features(n_nodes, density)
        if nx.is_connected(G):
            label, _, _ = evaluate_search_performance(G)
            X_train.append(features)
            y_train.append(label)
    except:
        continue

X_train = torch.FloatTensor(X_train)
y_train = torch.LongTensor(y_train)

# Trénování modelu
model = SearchStrategyPredictor()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()

print("\n🏋️ Trénování neuronové sítě...")
losses = []

for epoch in range(200):
    optimizer.zero_grad()
    outputs = model(X_train)
    loss = criterion(outputs, y_train)
    loss.backward()
    optimizer.step()
    
    losses.append(loss.item())
    
    if epoch % 40 == 0:
        accuracy = (outputs.argmax(1) == y_train).float().mean()
        print(f"Epocha {epoch}: Loss = {loss.item():.4f}, Přesnost = {accuracy:.2%}")

# Vizualizace trénování
plt.figure(figsize=(10, 5))
plt.plot(losses)
plt.title('Průběh trénování prediktoru strategie')
plt.xlabel('Epocha')
plt.ylabel('Loss')
plt.grid(True)
plt.show()

## 5. Transformer pro učení se prohledávání 🤖

Použijeme transformer architekturu pro učení se optimálních prohledávacích strategií.

In [ ]:
# Transformer agent pro prohledávání
class TransformerSearchAgent(nn.Module):
    def __init__(self, state_dim, action_dim, d_model=128, n_heads=4):
        super(TransformerSearchAgent, self).__init__()
        
        # Enkodér pro stavy grafu
        self.state_encoder = nn.Linear(state_dim, d_model)
        
        # Pozicový encoding
        self.pos_encoding = nn.Parameter(torch.randn(1, 100, d_model))
        
        # Transformer bloky
        encoder_layer = nn.TransformerEncoderLayer(
            d_model=d_model,
            nhead=n_heads,
            dim_feedforward=d_model * 4,
            batch_first=True
        )
        self.transformer = nn.TransformerEncoder(encoder_layer, num_layers=3)
        
        # Výstupní hlavy
        self.action_head = nn.Linear(d_model, action_dim)
        self.value_head = nn.Linear(d_model, 1)
    
    def forward(self, state_sequence):
        # Enkódování stavů
        seq_len = state_sequence.size(1)
        encoded = self.state_encoder(state_sequence)
        encoded += self.pos_encoding[:, :seq_len, :]
        
        # Transformer processing
        output = self.transformer(encoded)
        
        # Predikce akcí a hodnot
        actions = self.action_head(output)
        values = self.value_head(output)
        
        return actions, values

# Prostředí pro učení prohledávání
class GraphSearchEnvironment:
    def __init__(self, graph_size=10):
        self.graph_size = graph_size
        self.reset()
    
    def reset(self):
        # Generování nového grafu
        self.G = nx.erdos_renyi_graph(self.graph_size, 0.3)
        self.graph_dict = {n: list(self.G.neighbors(n)) for n in self.G.nodes()}
        
        # Náhodný start a cíl
        nodes = list(self.G.nodes())
        self.start = np.random.choice(nodes)
        self.goal = np.random.choice([n for n in nodes if n != self.start])
        
        # Aktuální stav
        self.current = self.start
        self.visited = {self.start}
        self.path = [self.start]
        
        return self.get_state()
    
    def get_state(self):
        """Vrátí reprezentaci aktuálního stavu"""
        state = np.zeros(self.graph_size * 3)
        
        # Aktuální pozice
        state[self.current] = 1
        
        # Cíl
        state[self.graph_size + self.goal] = 1
        
        # Navštívené uzly
        for v in self.visited:
            state[2 * self.graph_size + v] = 1
        
        return state
    
    def get_valid_actions(self):
        """Vrátí platné akce z aktuálního stavu"""
        neighbors = self.graph_dict[self.current]
        return neighbors + [self.current]  # Může zůstat na místě
    
    def step(self, action):
        """Provede akci a vrátí nový stav"""
        valid_actions = self.get_valid_actions()
        
        if action < len(valid_actions):
            self.current = valid_actions[action]
            self.visited.add(self.current)
            self.path.append(self.current)
        
        # Výpočet odměny
        if self.current == self.goal:
            reward = 100 - len(self.path)  # Bonus za kratší cestu
            done = True
        else:
            reward = -1  # Penalizace za každý krok
            done = len(self.path) > self.graph_size * 2  # Timeout
        
        return self.get_state(), reward, done

# Demonstrace transformer agenta
env = GraphSearchEnvironment()
agent = TransformerSearchAgent(state_dim=30, action_dim=10)

print("🤖 Transformer agent pro prohledávání grafů:")
print(f"Architektura: {sum(p.numel() for p in agent.parameters())} parametrů")

# Ukázka inference
state = env.reset()
state_tensor = torch.FloatTensor(state).unsqueeze(0).unsqueeze(0)

with torch.no_grad():
    actions, values = agent(state_tensor)
    print(f"\nPredikované akce: {actions.squeeze().softmax(0)}")
    print(f"Predikovaná hodnota stavu: {values.item():.2f}")

## 6. Interaktivní simulátor prohledávání 🎮

Vytvoříme Gradio aplikaci pro interaktivní experimentování s algoritmy.

In [ ]:
# Interaktivní simulátor
class SearchSimulator:
    def __init__(self):
        self.graph = None
        self.search_history = []
        self.neural_predictor = model  # Použijeme natrénovaný model
    
    def create_custom_graph(self, nodes_text, edges_text):
        """Vytvoří graf z textového vstupu"""
        try:
            nodes = [n.strip() for n in nodes_text.split(',')]
            edges = [e.strip().split('-') for e in edges_text.split(',')]
            
            self.graph = {node: [] for node in nodes}
            for edge in edges:
                if len(edge) == 2:
                    self.graph[edge[0]].append(edge[1])
                    self.graph[edge[1]].append(edge[0])
            
            return "Graf úspěšně vytvořen!"
        except Exception as e:
            return f"Chyba při vytváření grafu: {e}"
    
    def run_search(self, algorithm, start, goal):
        """Spustí vybraný algoritmus"""
        if not self.graph:
            return None, "Nejprve vytvořte graf!"
        
        if algorithm == "BFS":
            searcher = VisualBFS(self.graph)
        elif algorithm == "DFS":
            searcher = VisualDFS(self.graph)
        else:
            return None, "Neznámý algoritmus!"
        
        path, steps = searcher.search(start, goal)
        
        # Vytvoření vizualizace
        G = nx.Graph(self.graph)
        pos = nx.spring_layout(G, seed=42)
        
        fig, ax = plt.subplots(figsize=(10, 8))
        
        # Barvy uzlů
        node_colors = []
        for node in G.nodes():
            if node == start:
                node_colors.append('green')
            elif node == goal:
                node_colors.append('red')
            elif node in searcher.visited:
                node_colors.append('lightblue')
            else:
                node_colors.append('lightgray')
        
        # Kreslení grafu
        nx.draw(G, pos, ax=ax, node_color=node_colors, 
                node_size=1500, with_labels=True, font_size=16)
        
        # Zvýraznění cesty
        if path:
            path_edges = [(path[i], path[i+1]) for i in range(len(path)-1)]
            nx.draw_networkx_edges(G, pos, ax=ax, edgelist=path_edges, 
                                  edge_color='red', width=4)
        
        ax.set_title(f'{algorithm} - Cesta: {", ".join(path) if path else "Nenalezena"}', 
                    fontsize=16)
        
        plt.tight_layout()
        plt.savefig('search_result.png', dpi=150, bbox_inches='tight')
        plt.close()
        
        # Statistiky
        stats = f"""
        📊 Výsledky prohledávání:
        - Algoritmus: {algorithm}
        - Start: {start}, Cíl: {goal}
        - Počet kroků: {len(steps)}
        - Počet navštívených uzlů: {len(searcher.visited)}
        - Délka nalezené cesty: {len(path) if path else 'N/A'}
        - Cesta: {' → '.join(path) if path else 'Cesta nenalezena'}
        - Pořadí návštěvy: {', '.join(searcher.search_order[:10])}{'...' if len(searcher.search_order) > 10 else ''}
        """
        
        return 'search_result.png', stats
    
    def predict_best_algorithm(self, nodes_text, edges_text):
        """Použije neuronovou síť k predikci nejlepšího algoritmu"""
        try:
            # Vytvoření grafu
            self.create_custom_graph(nodes_text, edges_text)
            G = nx.Graph(self.graph)
            
            # Extrakce příznaků
            _, features = generate_graph_features(len(G.nodes()), nx.density(G))
            features_tensor = torch.FloatTensor(features).unsqueeze(0)
            
            # Predikce
            with torch.no_grad():
                output = self.neural_predictor(features_tensor)
                prediction = output.argmax(1).item()
                confidence = output.softmax(1).max().item()
            
            algorithm = "BFS" if prediction == 0 else "DFS"
            
            return f"🧠 Neuronová síť doporučuje: {algorithm} (jistota: {confidence:.1%})"
        except Exception as e:
            return f"Chyba při predikci: {e}"

# Vytvoření Gradio rozhraní
simulator = SearchSimulator()

def gradio_search(nodes, edges, algorithm, start, goal):
    simulator.create_custom_graph(nodes, edges)
    return simulator.run_search(algorithm, start, goal)

def gradio_predict(nodes, edges):
    return simulator.predict_best_algorithm(nodes, edges)

# Gradio aplikace
with gr.Blocks(title="Simulátor prohledávání grafů") as demo:
    gr.Markdown("# 🔍 Interaktivní simulátor BFS a DFS")
    
    with gr.Tab("Vytvoření grafu a prohledávání"):
        with gr.Row():
            with gr.Column():
                nodes_input = gr.Textbox(
                    label="Uzly (oddělené čárkou)",
                    value="A,B,C,D,E,F,G",
                    placeholder="A,B,C,D"
                )
                edges_input = gr.Textbox(
                    label="Hrany (formát: uzel1-uzel2, oddělené čárkou)",
                    value="A-B,A-C,B-D,B-E,C-F,E-F,F-G",
                    placeholder="A-B,B-C,C-D"
                )
                
                algorithm_select = gr.Radio(
                    choices=["BFS", "DFS"],
                    label="Algoritmus",
                    value="BFS"
                )
                
                start_input = gr.Textbox(
                    label="Počáteční uzel",
                    value="A"
                )
                
                goal_input = gr.Textbox(
                    label="Cílový uzel",
                    value="G"
                )
                
                search_btn = gr.Button("Spustit prohledávání", variant="primary")
            
            with gr.Column():
                result_image = gr.Image(label="Vizualizace")
                result_text = gr.Textbox(label="Výsledky", lines=10)
        
        search_btn.click(
            fn=gradio_search,
            inputs=[nodes_input, edges_input, algorithm_select, 
                   start_input, goal_input],
            outputs=[result_image, result_text]
        )
    
    with gr.Tab("Neuronová predikce"):
        gr.Markdown("### 🧠 Nechte neuronovou síť doporučit nejlepší algoritmus")
        
        with gr.Row():
            with gr.Column():
                pred_nodes = gr.Textbox(
                    label="Uzly grafu",
                    value="A,B,C,D,E,F,G,H"
                )
                pred_edges = gr.Textbox(
                    label="Hrany grafu",
                    value="A-B,A-C,B-D,C-E,D-F,E-G,F-H,G-H"
                )
                predict_btn = gr.Button("Získat doporučení", variant="primary")
            
            with gr.Column():
                prediction_output = gr.Textbox(
                    label="Doporučení neuronové sítě",
                    lines=3
                )
        
        predict_btn.click(
            fn=gradio_predict,
            inputs=[pred_nodes, pred_edges],
            outputs=prediction_output
        )
    
    gr.Markdown(
        """
        ### 📝 Návod:
        1. Zadejte uzly a hrany vašeho grafu
        2. Vyberte algoritmus (BFS nebo DFS)
        3. Určete počáteční a cílový uzel
        4. Spusťte prohledávání a sledujte výsledky
        5. Vyzkoušejte neuronovou predikci pro doporučení algoritmu
        """
    )

# Spuštění aplikace
print("🚀 Spouštím interaktivní simulátor...")
demo.launch(share=True)

## 7. Praktické příklady použití 🛠️

Aplikace BFS a DFS na reálné problémy.

In [ ]:
# Příklad 1: Hledání cesty v bludišti
class MazeSolver:
    def __init__(self, maze):
        self.maze = np.array(maze)
        self.rows, self.cols = self.maze.shape
    
    def get_neighbors(self, pos):
        row, col = pos
        neighbors = []
        
        # 4 směry: nahoru, vpravo, dolů, vlevo
        for dr, dc in [(-1, 0), (0, 1), (1, 0), (0, -1)]:
            new_row, new_col = row + dr, col + dc
            if (0 <= new_row < self.rows and 
                0 <= new_col < self.cols and 
                self.maze[new_row, new_col] == 0):
                neighbors.append((new_row, new_col))
        
        return neighbors
    
    def solve_bfs(self, start, end):
        queue = deque([start])
        visited = {start}
        parent = {start: None}
        
        while queue:
            current = queue.popleft()
            
            if current == end:
                # Rekonstrukce cesty
                path = []
                while current is not None:
                    path.append(current)
                    current = parent[current]
                return path[::-1]
            
            for neighbor in self.get_neighbors(current):
                if neighbor not in visited:
                    visited.add(neighbor)
                    parent[neighbor] = current
                    queue.append(neighbor)
        
        return None
    
    def visualize_solution(self, path):
        solution_maze = self.maze.copy()
        
        # Označení cesty
        for row, col in path:
            solution_maze[row, col] = 2
        
        # Vizualizace
        plt.figure(figsize=(8, 8))
        plt.imshow(solution_maze, cmap='RdYlBu')
        plt.title('Řešení bludiště pomocí BFS')
        plt.colorbar(label='0=cesta, 1=zeď, 2=řešení')
        plt.show()

# Vytvoření bludiště
maze = [
    [0, 1, 0, 0, 0, 0, 0, 0],
    [0, 1, 0, 1, 1, 1, 1, 0],
    [0, 0, 0, 0, 0, 0, 1, 0],
    [1, 1, 1, 1, 1, 0, 1, 0],
    [0, 0, 0, 0, 0, 0, 1, 0],
    [0, 1, 1, 1, 1, 1, 1, 0],
    [0, 0, 0, 0, 0, 0, 0, 0],
    [0, 1, 1, 1, 1, 1, 1, 0]
]

solver = MazeSolver(maze)
path = solver.solve_bfs((0, 0), (7, 7))

print("🏃 Řešení bludiště:")
if path:
    print(f"Cesta nalezena! Délka: {len(path)} kroků")
    solver.visualize_solution(path)
else:
    print("Cesta nenalezena!")

# Příklad 2: Detekce cyklů v grafu pomocí DFS
def has_cycle_dfs(graph):
    """Detekuje cykly v neorientovaném grafu"""
    visited = set()
    
    def dfs(node, parent):
        visited.add(node)
        
        for neighbor in graph[node]:
            if neighbor not in visited:
                if dfs(neighbor, node):
                    return True
            elif parent != neighbor:
                return True
        
        return False
    
    # Kontrola všech komponent
    for node in graph:
        if node not in visited:
            if dfs(node, None):
                return True
    
    return False

# Test detekce cyklů
graph_with_cycle = {
    'A': ['B', 'C'],
    'B': ['A', 'C'],
    'C': ['A', 'B', 'D'],
    'D': ['C']
}

graph_without_cycle = {
    'A': ['B'],
    'B': ['A', 'C', 'D'],
    'C': ['B'],
    'D': ['B']
}

print("\n🔄 Detekce cyklů:")
print(f"Graf s cyklem: {has_cycle_dfs(graph_with_cycle)}")
print(f"Graf bez cyklu: {has_cycle_dfs(graph_without_cycle)}")

## 8. Praktická cvičení 📝

Vyzkoušejte si implementaci vlastních variant algoritmů!

In [ ]:
# Cvičení 1: Implementujte iterativní DFS
def iterative_dfs(graph, start, goal):
    """
    TODO: Implementujte iterativní verzi DFS pomocí zásobníku
    Vraťte cestu od startu k cíli nebo None
    """
    # Váš kód zde
    pass

# Cvičení 2: BFS pro nejkratší cestu s váhami
def weighted_bfs(graph, weights, start, goal):
    """
    TODO: Upravte BFS pro grafy s váhami hran
    Tip: Použijte prioritní frontu (heapq)
    """
    # Váš kód zde
    pass

# Cvičení 3: Bidirectional search
def bidirectional_search(graph, start, goal):
    """
    TODO: Implementujte obousměrné prohledávání
    Spusťte BFS ze startu i cíle současně
    """
    # Váš kód zde
    pass

# Cvičení 4: Neuronová heuristika pro vedené prohledávání
class NeuralGuidedSearch(nn.Module):
    """
    TODO: Vytvořte neuronovou síť, která předpovídá
    vzdálenost uzlu od cíle pro vedené prohledávání
    """
    def __init__(self):
        super(NeuralGuidedSearch, self).__init__()
        # Váš kód zde
        pass
    
    def forward(self, node_features):
        # Váš kód zde
        pass

print("📚 Cvičení připravena! Implementujte řešení výše.")
print("\nTipy:")
print("- Pro cvičení 1: Použijte list jako zásobník (append/pop)")
print("- Pro cvičení 2: heapq.heappush a heappop pro prioritní frontu")
print("- Pro cvičení 3: Udržujte dvě fronty a kontrolujte průnik")
print("- Pro cvičení 4: Vstup může být one-hot encoding uzlu + cíle")

## 9. Shrnutí a klíčové koncepty 🎓

### Co jsme se naučili:

1. **BFS (Breadth-First Search)**
   - Prohledává do šířky, úroveň po úrovni
   - Garantuje nejkratší cestu v neohodnocených grafech
   - Používá frontu (FIFO)

2. **DFS (Depth-First Search)**
   - Prohledává do hloubky
   - Paměťově efektivnější
   - Používá zásobník (LIFO)

3. **Neuronové sítě pro prohledávání**
   - Predikce optimální strategie
   - Učení se heuristik
   - Transformer architektury pro plánování

4. **Praktické aplikace**
   - Řešení bludišť
   - Detekce cyklů
   - Hledání cest v grafech

### Kdy použít který algoritmus:

- **BFS**: Když potřebujete nejkratší cestu, máte dostatek paměti
- **DFS**: Když je graf hluboký, máte omezenu paměť, nebo hledáte jakoukoliv cestu

### Další kroky:
- V další hodině se podíváme na **informované prohledávání** (A*, heuristiky)
- Naučíme se kombinovat klasické algoritmy s neuronovými sítěmi

In [ ]:
# Závěrečný interaktivní kvíz
print("🎉 Gratulujeme! Dokončili jste hodinu o neinformovaném prohledávání!")
print("\n📊 Vaše pokroky:")
print("✅ Implementace BFS a DFS")
print("✅ Vizualizace průběhu algoritmů")
print("✅ Neuronové sítě pro predikci strategií")
print("✅ Transformer architektury pro prohledávání")
print("✅ Praktické aplikace na reálné problémy")

# Mini srovnání algoritmů
comparison_data = {
    'Vlastnost': ['Datová struktura', 'Kompletnost', 'Optimalita', 
                  'Časová složitost', 'Paměťová složitost'],
    'BFS': ['Fronta', 'Ano', 'Ano*', 'O(b^d)', 'O(b^d)'],
    'DFS': ['Zásobník', 'Ne**', 'Ne', 'O(b^m)', 'O(bm)']
}

import pandas as pd
df = pd.DataFrame(comparison_data)

print("\n📋 Srovnání BFS a DFS:")
print(df.to_string(index=False))
print("\n* Pro neohodnocené grafy")
print("** V nekonečných prostorech")
print("b = branching factor, d = hloubka řešení, m = max hloubka")

print("\n🚀 Připraveni na informované prohledávání v další hodině!")

### Hodina 12 — Prohledávání do šířky (BFS) — ELI10

Představ si, že stojíš v jedné místnosti v domě a chceš najít všechny místnosti, které jsou nejblíže k tobě, než půjdeš dál do vzdálenějších míst. BFS (Breadth‑First Search) dělá to samé: nejdřív prohledá všechny sousedy aktuálního stavu, pak jejich sousedy, a tak postupně dál. Je to užitečné, když chceme najít nejkratší cestu v neohodnoceném grafu.

Níže je jednoduchý a spustitelný příklad BFS na malém grafu.

In [ ]:
# BFS example on an adjacency list
from collections import deque

def bfs(graph, start):
    q = deque([start])
    visited = {start}
    order = []
    while q:
        node = q.popleft()
        order.append(node)
        for nbr in graph.get(node, []):
            if nbr not in visited:
                visited.add(nbr)
                q.append(nbr)
    return order

# Small test graph
G = {
    'A': ['B','C'],
    'B': ['D','E'],
    'C': ['F'],
    'D': [],
    'E': [],
    'F': []
}

order = bfs(G, 'A')
print('BFS order:', order)
# Sanity checks
assert order[0] == 'A'
assert set(order) == set(G.keys())
# Expected BFS order begins with A then B and C (level 1)
assert order[1] in ('B','C')
print('BFS checks passed')

Úkoly:

1) Implementujte DFS (do hloubky) a porovnejte pořadí průchodu na tomtéž grafu.
2) Vytvořte graf s cyklem a ověřte, že BFS cyklus nezpůsobí nekonečný smyčku (použijte visited set).
3) Bonus: Vykreslete graf pomocí `networkx` a `matplotlib` a zvýrazněte cestu nalezenou BFS.